In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27660105
paper_name = 'kwon_kwak_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table1.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 23 x 6


In [7]:
original_data.head()

,ORF,Gene,Log2 ratio (14 μM),Fitness score (W),SGD description,Pool
0,YJR068W,RFC2,6.44,0.51,Subunit of heteropentameric replication factor...,HIP
1,YKR038C,KAE1,5.82,0.73,Highly conserved ATPase of HSP70/DnaK family; ...,HIP
2,YJR121W,ATP2,5.79,0.84,Beta subunit of the F1 sector of mitochondrial...,HIP
3,YNL322C,KRE1,5.73,0.88,Cell wall glycoprotein involved in beta-glucan...,HIP
4,YGL263W,COS12,5.61,1.01,"Protein of unknown function, member of the DUP...",HIP


In [9]:
original_data['orf'] = original_data['ORF '].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF , Gene , Log2 ratio (14 μM) , Fitness score (W) , SGD description , Pool , orf]
Index: []


In [14]:
original_data['Pool '].unique()

array(['HIP ', 'HOP '], dtype=object)

In [16]:
original_data['data'] = original_data['Fitness score (W) ']

In [18]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data1 = original_data.loc[original_data['Pool ']=='HOP ',['data']].copy()
original_data2 = original_data.loc[original_data['Pool ']=='HIP ',['data']].copy()

In [22]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

In [23]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [24]:
original_data.shape

(23, 2)

In [25]:
original_data.head()

,data_1,data_2
orf,,
YBR211C,NaN,0.66
YEL011W,NaN,0.78
YER133W,NaN,0.65
YGL219C,0.76,NaN
YGL263W,NaN,1.01


# Prepare the final dataset

In [26]:
data = original_data.copy()

In [27]:
dataset_ids = [16140, 16143]
datasets = datasets.reindex(index=dataset_ids)

In [28]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [29]:
data.head()

dataset_id,16140,16143
data_type,value,value
orf,,
YBR211C,NaN,0.66
YEL011W,NaN,0.78
YER133W,NaN,0.65
YGL219C,0.76,NaN
YGL263W,NaN,1.01


## Subset to the genes currently in SGD

In [30]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [31]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [32]:
data.head()

,dataset_id,16140,16143
,data_type,value,value
gene_id,orf,,
405,YBR211C,NaN,0.66
666,YEL011W,NaN,0.78
862,YER133W,NaN,0.65
2900,YGL219C,0.76,NaN
2945,YGL263W,NaN,1.01


# Normalize

In [33]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [34]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [35]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [36]:
data_all.head()


,dataset_id,16140,16143,16140,16143
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
405,YBR211C,NaN,0.66,NaN,13.422630
666,YEL011W,NaN,0.78,NaN,15.863109
862,YER133W,NaN,0.65,NaN,13.219257
2900,YGL219C,0.76,NaN,34.571501,NaN
2945,YGL263W,NaN,1.01,NaN,20.540692


# Print out

In [37]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [38]:
from IO.save_data_to_db3 import *

In [39]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 27660105...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00, 10.04it/s]

Updating the data_modified_on field...
